In [1]:
# Step 1: Install necessary libraries
!pip install -q transformers datasets accelerate bitsandbytes torch evaluate rouge_score sentencepiece bert_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.5 MB/s eta 0:00:00
   ━

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from datasets import load_dataset
from kaggle_secrets import UserSecretsClient
import pandas as pd
import random
import evaluate # Hugging Face's library for NLP evaluation
import warnings
import time

# Suppress warnings to keep the output clean
warnings.filterwarnings("ignore")

2025-09-12 20:28:38.991667: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757708919.240844      18 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757708919.319546      18 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Step 2: Authenticate with Hugging Face
# This is required to download gated models like Llama 3
try:
    user_secrets = UserSecretsClient()
    hf_token = user_secrets.get_secret("HUGGING_FACE_TOKEN")
except Exception as e:
    print("Could not retrieve Hugging Face token. Please ensure it is stored as a Kaggle secret named 'HUGGING_FACE_TOKEN'.")
    # You can manually paste your token here for local testing if needed:
    # hf_token = "YOUR_HF_TOKEN"
    hf_token = None

In [4]:
# Step 3: Define Model and Dataset Identifiers
dataset_id = "tmnam20/ViMedAQA"
NUM_SAMPLES_TO_EVALUATE = 200
model_ids = [
    "alpha-ai/LLAMA3-3B-Medical-COT",
    "vilm/vietcuna-3b-v2",
    "arcee-ai/Arcee-VyLinh",
]

# Step 4: Load and Prepare the Dataset
try:
    dataset = load_dataset(dataset_id, split="train")
    print(f"Dataset loaded successfully! Total samples: {len(dataset)}")

    # Create a small, random, representative sample for evaluation
    random.seed(42) # for reproducibility
    random_indices = random.sample(range(len(dataset)), NUM_SAMPLES_TO_EVALUATE)
    eval_dataset = dataset.select(random_indices)

    print(f"Created a random evaluation set with {len(eval_dataset)} samples.")
except Exception as e:
    print(f"Failed to load the dataset. Error: {e}")
    eval_dataset = None

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/20.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/39881 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2217 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2215 [00:00<?, ? examples/s]

Dataset loaded successfully! Total samples: 39881
Created a random evaluation set with 200 samples.


In [5]:
# --- UPDATED: Step 1 - Define Bilingual Prompt Engineering Strategies ---
# Mode 1: Run all strategies defined in PROMPT_STRATEGIES (False)
# Mode 2: Run only the single, specified strategy for a targeted comparison (True)
USE_BEST_PROMPT_ONLY = False
BEST_STRATEGY_NAME = "Current_Best_VI" # Specify the prompt to use in Mode 2
generation_times = {}

PROMPT_STRATEGIES = {
    # Các chiến lược ban đầu của bạn
    "Direct_VI": "Sử dụng Ngữ cảnh sau để trả lời Câu hỏi.",
    "RolePlay_VI": "Bạn là một trợ lý y tế hữu ích. Hãy trả lời Câu hỏi CHỈ dựa vào Ngữ cảnh được cung cấp.",
    "Extract_VI": "Dựa vào Ngữ cảnh sau, trích xuất câu trả lời trực tiếp từ văn bản, không giải thích gì thêm.",
    "Current_Best_VI": (
        "Bạn là một chuyên gia y tế AI với nhiệm vụ trích xuất thông tin chính xác. "
        "Dựa CHỈ vào văn bản trong phần Ngữ cảnh dưới đây, hãy trả lời cho Câu hỏi. "
        "Câu trả lời của bạn phải ngắn gọn, đi thẳng vào vấn đề và không chứa bất kỳ thông tin nào không có trong văn bản. "
        "Không giải thích thêm."
    ),

    # Các chiến lược được thêm vào
    "Chain_of_Thought_VI": (
        "Dựa vào Ngữ cảnh sau, hãy suy nghĩ từng bước một để đưa ra câu trả lời cho Câu hỏi. "
        "Hãy trình bày rõ ràng các bước suy luận của bạn."
    ),
    "Few_Shot_VI_ViMedAQA": (
        "Dựa vào các Ví dụ sau đây, hãy trả lời Câu hỏi cuối cùng bằng cách trích xuất thông tin từ Ngữ cảnh được cung cấp.\n\n"
        "--- Ví dụ 1 ---\n"
        "Ngữ cảnh: Thuốc Biviantac được chỉ định để điều trị các trường hợp do tăng tiết acid quá mức như: - Khó tiêu, nóng rát hay đau vùng thượng vị. - Trướng bụng, đầy hơi, ợ nóng, ợ hơi hay ợ chua. - Tăng độ acid, đau rát dạ dày. - Các rối loạn thường gặp trong những bệnh lý loét dạ dày tá tràng, thực quản.\n"
        "Câu hỏi: Biviantac có thể điều trị trướng bụng, đầy hơi không?\n"
        "Câu trả lời: Có, Biviantac có thể điều trị các tình trạng như trướng bụng, đầy hơi, ợ nóng, ợ hơi hay ợ chua.\n\n"
        "--- Ví dụ 2 ---\n"
        "Ngữ cảnh: Thuốc Atorvastatin T.V Pharm được dùng đường uống.\n"
        "Câu hỏi: Tổng hợp các cách dùng hiệu quả để quản lý Atorvastatin T.V Pharm?\n"
        "Câu trả lời: Các cách thức dùng thuốc Atorvastatin T.V Pharm hiệu quả là sử dụng đường uống.\n\n"
        "--- Ví dụ 3 ---\n"
        "Ngữ cảnh: - Buồn nôn, nôn, khó tiêu, khó chịu ở thượng vị, ợ nóng, đau dạ dày, loét dạ dày – ruột. - Mệt mỏi. - Ban, mày đay. - Thiếu máu tan huyết. - Yếu cơ. - Khó thở, sốc phản vệ.\n"
        "Câu hỏi: Các tác dụng phụ thường gặp của thuốc Aspirin 81 là gì?\n"
        "Câu trả lời: Các tác dụng phụ thường gặp của thuốc Aspirin 81 bao gồm buồn nôn, nôn, khó tiêu, khó chịu ở thượng vị, ợ nóng, đau dạ dày, loét dạ dày – ruột.\n\n"
        "--- Bây giờ, hãy trả lời Câu hỏi sau dựa trên Ngữ cảnh của nó---\n"
    ),
    "Expert_Persona_VI": (
        "Bạn là một chuyên gia trong lĩnh vực y tế."
        "Dựa trên kiến thức chuyên môn của mình, hãy trả lời Câu hỏi sau CHỈ dựa vào Ngữ cảnh được cung cấp."
    ),
}


def create_prompt(sample, model_id, tokenizer, strategy_name):
    """
    A single, unified function to create a fairly-compared prompt.

    It takes a strategy name, finds the corresponding instruction text from the
    central dictionary, and then wraps it in the correct format for the target model.
    """
    context = sample['context']
    question = sample['question']

    # 1. Get the unified instruction text for the chosen strategy
    base_instruction = PROMPT_STRATEGIES.get(strategy_name)
    if not base_instruction:
        raise ValueError(f"Strategy '{strategy_name}' not found in PROMPT_STRATEGIES dictionary.")

    full_instruction_text = f"{base_instruction}\n\nNgữ cảnh: {context}\n\nCâu hỏi: {question}"

    # 2. Apply the correct, model-specific formatting
    if 'vietcuna' in model_id:
        return f"A chat between a curious user and an artificial intelligence assistant.\nUSER: {full_instruction_text}\nASSISTANT:"
        # return f"Một cuộc trò chuyện giữa một người dùng tò mò và một trợ lý trí tuệ nhân tạo.\nUSER: {full_instruction_text}\nASSISTANT:"

    elif 'Arcee-VyLinh' in model_id or 'LLAMA3' in model_id:
        messages = [
            {"role": "system", "content": "Bạn là một trợ lý y tế hữu ích. Hãy trả lời Câu hỏi của người dùng CHỈ dựa vào Ngữ cảnh được cung cấp."},
            {"role": "user", "content": full_instruction_text}
        ]
        return tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

    else: 
        # Default to Llama 3 format
        # return f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{full_instruction_text}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
        print("ERROR: No model found")
        return ""

In [6]:
# Step 5: Generate Answers from Each Model
all_generated_answers = {}

if eval_dataset and hf_token:
    # Wrap each answer in a list to create the required List[List[str]] structure
    ground_truth_answers = [[sample['answer']] for sample in eval_dataset] 
    questions = [sample['question'] for sample in eval_dataset]

    wide_results = []
    for i, sample in enumerate(eval_dataset):
        wide_results.append({
            "Sample_ID": i,
            "Question": sample['question'],
            "Context": sample['context'],
            "Ground_Truth_Answer": ground_truth_answers[i][0]
        })
    
    # Loop through each model to generate answers
    for model_id in model_ids:
        print("\n" + "="*50)
        print(f"Loading model: {model_id}")
        print("="*50)

        model, tokenizer, text_generator = None, None, None

        try:
            # Load the tokenizer and model with 4-bit quantization to save memory
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16,
                bnb_4bit_use_double_quant=False,
            )
            tokenizer = AutoTokenizer.from_pretrained(model_id, token=hf_token)
            model = AutoModelForCausalLM.from_pretrained(
                model_id,
                token=hf_token,
                quantization_config=bnb_config, # <-- PASS THE CONFIG OBJECT HERE
                device_map="auto",
                trust_remote_code=True
            )

            # Set up the text generation pipeline
            text_generator = pipeline(
                "text-generation",
                model=model,
                tokenizer=tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
            )

            # Determine which set of prompt functions to use
            if 'vietcuna' in model_id:
                answer_start_tag = "ASSISTANT:"
            elif 'Arcee-VyLinh' in model_id:
                answer_start_tag = "<|im_start|>assistant"
            else: # Default to Llama3
                answer_start_tag = "<|start_header_id|>assistant<|end_header_id|>"

            prompt_variations = []
            if USE_BEST_PROMPT_ONLY:
                # Mode 2: Run only the single specified prompt strategy
                prompt_variations = [BEST_STRATEGY_NAME]
                print(f"Mode: Best Prompt Only. Running with the fair strategy: '{BEST_STRATEGY_NAME}'")
            else:
                # Mode 1: Run all defined prompt strategies for a full exploration
                prompt_variations = list(PROMPT_STRATEGIES.keys())
                print(f"Mode: Exploration. Running all {len(prompt_variations)} fair strategies: {prompt_variations}")

                
            for prompt_name in prompt_variations:
                print(f"\n--- Testing Prompt Strategy: {prompt_name} ---")
                # Create a unique key for each result set
                result_key = f"{model_id} ({prompt_name})"
                
                # Generate all prompts using the single, fair prompt factory function
                prompts = [create_prompt(sample, model_id, tokenizer, prompt_name) for sample in eval_dataset]
                # print(prompts)

                start_time = time.time()

                print(f"Generating answers for {len(prompts)} prompts using {model_id} with '{prompt_name}' strategy...")
                # Generate answers for the entire batch
                generated_outputs_batch = text_generator(
                    prompts,
                    max_new_tokens=256,
                    do_sample=False,      # Tắt việc lấy mẫu
                    # temperature=0.0,      # Đặt temperature về 0 (thường được đặt cùng với do_sample=False)
                    # # top_p không còn tác dụng khi do_sample=False, nhưng để lại cũng không sao
                    # top_p=0.9,
                    eos_token_id=tokenizer.eos_token_id,
                    pad_token_id=tokenizer.eos_token_id,
                )

                end_time = time.time()
                generation_time = end_time - start_time
                generation_times[result_key] = generation_time # Lưu thời gian đã đo
                print(f"Time for generating answer: {generation_time:.2f} seconds.")
    
                # Extract the clean answers
                model_answers = []
                
                for i, output in enumerate(generated_outputs_batch):
                    generated_text = output[0]['generated_text']
                    # Use the appropriate start tag for splitting
                    if answer_start_tag in generated_text:
                        clean_answer = generated_text.split(answer_start_tag)[-1].strip()
                    else:
                        # Fallback remains the same
                        clean_answer = generated_text.replace(prompts[i], "").strip()
                    model_answers.append(clean_answer)
    
                # Use the unique result_key to store the answers
                all_generated_answers[result_key] = model_answers
                print(f"Successfully generated answers for {result_key}.")

                # --- Thêm các câu trả lời đã tạo như một cột mới vào cấu trúc "rộng" ---
                # Tên cột sẽ là nhãn của prompt, ví dụ: "Answer_vilm/vietcuna-3b-v2 (Current_Best_VI)"
                answer_column_name = f"Answer_{result_key}"
                
                # Lặp qua các câu trả lời và thêm chúng vào đúng hàng trong wide_results
                for i in range(len(model_answers)):
                    wide_results[i][answer_column_name] = model_answers[i]

        except Exception as e:
            print(f"An error occurred while processing {model_id}: {e}")
        finally:
            # Check if variables were successfully created before deleting
            if model is not None: del model
            if tokenizer is not None: del tokenizer
            if text_generator is not None: del text_generator
            torch.cuda.empty_cache()

            import gc
            gc.collect()

else:
    print("Skipping generation due to issues with the dataset or Hugging Face token.")


Loading model: alpha-ai/LLAMA3-3B-Medical-COT


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/982 [00:00<?, ?B/s]

...

Time for generating answer: 4570.40 seconds.
Successfully generated answers for arcee-ai/Arcee-VyLinh (Chain_of_Thought_VI).

--- Testing Prompt Strategy: Few_Shot_VI_ViMedAQA ---
Generating answers for 200 prompts using arcee-ai/Arcee-VyLinh with 'Few_Shot_VI_ViMedAQA' strategy...


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignore

Time for generating answer: 1777.16 seconds.
Successfully generated answers for arcee-ai/Arcee-VyLinh (Few_Shot_VI_ViMedAQA).

--- Testing Prompt Strategy: Expert_Persona_VI ---
Generating answers for 200 prompts using arcee-ai/Arcee-VyLinh with 'Expert_Persona_VI' strategy...


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignore

Time for generating answer: 2283.51 seconds.
Successfully generated answers for arcee-ai/Arcee-VyLinh (Expert_Persona_VI).


In [7]:
# --- Bước 5.5 - Lưu tất cả các câu trả lời đã tạo vào một tệp CSV ---
if wide_results:
    print("\n" + "="*50)
    print("Đang lưu kết quả định dạng rộng vào tệp CSV...")
    print("="*50)
    
    # Chuyển đổi danh sách kết quả thành một DataFrame của pandas
    results_df_wide = pd.DataFrame(wide_results)
    
    # Chỉ định đường dẫn tệp đầu ra trong thư mục làm việc của Kaggle
    output_file_path = "/kaggle/working/generation_results_wide.csv"
    
    # Lưu DataFrame vào tệp CSV
    results_df_wide.to_csv(output_file_path, index=False, encoding='utf-8-sig')
    
    print(f"Hoàn tất! Đã lưu {len(results_df_wide)} hàng (mẫu) vào tệp:")
    print(output_file_path)
    
    # Hiển thị 5 hàng đầu tiên của tệp đã lưu để xem trước
    # Bạn sẽ thấy các cột câu trả lời khác nhau cho mỗi chiến lược prompt
    display(results_df_wide.head())
    
else:
    print("\nKhông có kết quả nào để lưu.")


Đang lưu kết quả định dạng rộng vào tệp CSV...
Hoàn tất! Đã lưu 200 hàng (mẫu) vào tệp:
/kaggle/working/generation_results_wide.csv


,Sample_ID,Question,Context,Ground_Truth_Answer,Answer_alpha-ai/LLAMA3-3B-Medical-COT (Direct_VI),Answer_alpha-ai/LLAMA3-3B-Medical-COT (RolePlay_VI),Answer_alpha-ai/LLAMA3-3B-Medical-COT (Extract_VI),Answer_alpha-ai/LLAMA3-3B-Medical-COT (Current_Best_VI),Answer_alpha-ai/LLAMA3-3B-Medical-COT (Chain_of_Thought_VI),Answer_alpha-ai/LLAMA3-3B-Medical-COT (Few_Shot_VI_ViMedAQA),...,Answer_vilm/vietcuna-3b-v2 (Chain_of_Thought_VI),Answer_vilm/vietcuna-3b-v2 (Few_Shot_VI_ViMedAQA),Answer_vilm/vietcuna-3b-v2 (Expert_Persona_VI),Answer_arcee-ai/Arcee-VyLinh (Direct_VI),Answer_arcee-ai/Arcee-VyLinh (RolePlay_VI),Answer_arcee-ai/Arcee-VyLinh (Extract_VI),Answer_arcee-ai/Arcee-VyLinh (Current_Best_VI),Answer_arcee-ai/Arcee-VyLinh (Chain_of_Thought_VI),Answer_arcee-ai/Arcee-VyLinh (Few_Shot_VI_ViMedAQA),Answer_arcee-ai/Arcee-VyLinh (Expert_Persona_VI)
0,0,D-Cure có tác dụng dự phòng những bệnh gì?,Thuốc D-Cure là sản phẩm có chứa hàm lượng vit...,D-Cure có tác dụng dự phòng tình trạng thiếu h...,"D-Cure là một sản phẩm có chứa vitamin D, giúp...",Tôi không thể cung cấp thông tin về tác dụng c...,- Bị bệnh loãng xương.,D-Cure có tác dụng dự phòng bệnh loãng xương.,"Để trả lời câu hỏi này, tôi sẽ suy nghĩ từng b...","Dựa vào Ngữ cảnh, câu trả lời cho câu hỏi cuối...",...,D-Cure có tác dụng dự phòng các bệnh như tình ...,D-Cure có tác dụng dự phòng các bệnh như bệnh ...,D-Cure có tác dụng dự phòng các bệnh như tình ...,D-Cure có tác dụng dự phòng những bệnh như thi...,D-Cure có tác dụng dự phòng bệnh loãng xương.,D-Cure có tác dụng dự phòng bệnh loãng xương.,D-Cure có tác dụng dự phòng bệnh loãng xương v...,"Để trả lời câu hỏi này, tôi sẽ phân tích từng ...",D-Cure có tác dụng dự phòng những bệnh như:\n\...,D-Cure có tác dụng dự phòng những bệnh như tìn...
1,1,Điều quan trọng cần làm khi dùng quá liều Atid...,Trong các trường hợp có dấu hiệu bất thường do...,Đi đến cơ sở y tế gần nhất hoặc liên hệ Trung ...,"Được chứ, bạn đã hỏi điều quan trọng cần làm k...","Khi dùng quá liều Atidogrel, người dùng cần th...",Độ liều Atidogrel quá mức có thể dẫn đến tình ...,"Khi dùng quá liều Atidogrel, cần ngừng sử dụng...",Dưới đây là các bước suy luận để đưa ra câu tr...,"Câu trả lời: Khi dùng quá liều Atidogrel, cần ...",...,"Khi dùng quá liều Atidogrel, bạn cần đến ngay ...",- Đến ngay cơ sở y tế gần nhất hoặc Trung tâm ...,"Khi dùng quá liều Atidogrel, điều quan trọng l...","Khi dùng quá liều Atorvastatin (Atidogrel), đi...","Khi dùng quá liều Atorvastatin (Atidogrel), đi...",Điều quan trọng cần làm khi dùng quá liều Atid...,Điều quan trọng cần làm khi dùng quá liều Atid...,"Dựa trên ngữ cảnh được cung cấp, tôi sẽ phân t...",Điều quan trọng cần làm khi dùng quá liều Atid...,"Khi dùng quá liều Atorvastatin (Atidogrel), đi..."
2,2,Nước Hương nhu được sử dụng như thế nào?,Tác dụng dược lý chống lại Streptococcus mutan...,Nước Hương nhu có thể được sử dụng như một loạ...,Nước Hương nhu có thể được sử dụng như một loạ...,Người dùng có thể uống nước Hương nhu để giúp ...,Nước Hương nhu được sử dụng như một loại nước ...,Nước Hương nhu được sử dụng như một loại nước ...,Dưới đây là các bước suy luận để đưa ra câu tr...,"Dựa vào Ngữ cảnh, câu trả lời cho câu hỏi của ...",...,Nước Hương nhu được sử dụng như một loại nước ...,Nước Hương nhu được sử dụng như một loại nước ...,Nước Hương nhu được sử dụng như một loại nước ...,Nước Hương nhu được sử dụng như một loại nước ...,Nước Hương nhu được sử dụng như một loại nước ...,Nước Hương nhu được sử dụng như một loại nước ...,Nước Hương nhu được sử dụng như một loại nước ...,"Để trả lời câu hỏi, tôi sẽ phân tích từng phần...",Nước Hương nhu được sử dụng như một loại nước ...,Nước Hương nhu được sử dụng như một loại nước ...
3,3,Dầu mù u có thể gây ra tác dụng phụ gì?,Dầu mù u được xem là an toàn khi sử dụng tại c...,Dầu mù u có thể gây dị ứng ở một số người.,Tôi không thể cung cấp thông tin về tác dụng p...,Tôi không thể cung cấp thông tin về tác dụng p...,Nguyên nhân gây ra các tác dụng phụ: \n\n- Ngứ...,Dầu mù u 

In [8]:
# Step 6: Evaluate the Generated Answers
if all_generated_answers:
    # Load all the metrics we need
    rouge_metric = evaluate.load('rouge')
    bleu_metric = evaluate.load('bleu')
    meteor_metric = evaluate.load('meteor')
    bertscore_metric = evaluate.load('bertscore')

    evaluation_results = []

    print("\n" + "="*50)
    print("Calculating Evaluation Metrics")
    print("="*50)

    for result_key, predictions in all_generated_answers.items():
        print(f"\n--- Evaluating {result_key} ---")
    
        # Check for empty predictions to prevent ZeroDivisionError in BLEU ---
        # The `any()` function returns False if all strings in the list are empty.
        if not any(predictions):
            print(f"  WARNING: Model & Prompt Strategy '{result_key}' produced empty answers for all samples. Assigning all metric scores to 0.")
            result_row = {
                "Model & Prompt Strategy": result_key,
                "ROUGE-L": 0.0,
                "BLEU": 0.0,
                "METEOR": 0.0,
                "BERTScore-F1": 0.0
            }
            evaluation_results.append(result_row)
            # Use `continue` to skip the rest of the loop and move to the next model
            continue
    
        # If predictions are valid, compute metrics as normal
        rouge_scores = rouge_metric.compute(predictions=predictions, references=ground_truth_answers)
        bleu_scores = bleu_metric.compute(predictions=predictions, references=ground_truth_answers)
        meteor_scores = meteor_metric.compute(predictions=predictions, references=ground_truth_answers)
        bertscore_scores = bertscore_metric.compute(predictions=predictions, references=ground_truth_answers, lang="vi")
    
        # Store results (this part is the same as before)
        result_row = {
            "Model & Prompt Strategy": result_key,
            "ROUGE-L": round(rouge_scores['rougeL'], 4),
            "BLEU": round(bleu_scores['bleu'], 4),
            "METEOR": round(meteor_scores['meteor'], 4),
            "BERTScore-F1": round(sum(bertscore_scores['f1']) / len(bertscore_scores['f1']), 4),
            "Generation Time (s)": round(generation_times.get(result_key, 0), 2), # Lấy thời gian đã lưu
        }
        evaluation_results.append(result_row)

    # Step 7: Display Results
    results_df = pd.DataFrame(evaluation_results)
    # Sort for better comparison
    results_df = results_df.sort_values(by="BERTScore-F1", ascending=False).reset_index(drop=True)
    print("\n--- Comparative Evaluation Results ---")
    display(results_df)

else:
    print("\nNo answers were generated. Skipping evaluation.")

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...



Calculating Evaluation Metrics

--- Evaluating alpha-ai/LLAMA3-3B-Medical-COT (Direct_VI) ---


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]


--- Evaluating alpha-ai/LLAMA3-3B-Medical-COT (RolePlay_VI) ---

--- Evaluating alpha-ai/LLAMA3-3B-Medical-COT (Extract_VI) ---

--- Evaluating alpha-ai/LLAMA3-3B-Medical-COT (Current_Best_VI) ---

--- Evaluating alpha-ai/LLAMA3-3B-Medical-COT (Chain_of_Thought_VI) ---

--- Evaluating alpha-ai/LLAMA3-3B-Medical-COT (Few_Shot_VI_ViMedAQA) ---

--- Evaluating alpha-ai/LLAMA3-3B-Medical-COT (Expert_Persona_VI) ---

--- Evaluating vilm/vietcuna-3b-v2 (Direct_VI) ---

--- Evaluating vilm/vietcuna-3b-v2 (RolePlay_VI) ---

--- Evaluating vilm/vietcuna-3b-v2 (Extract_VI) ---

--- Evaluating vilm/vietcuna-3b-v2 (Current_Best_VI) ---

--- Evaluating vilm/vietcuna-3b-v2 (Chain_of_Thought_VI) ---

--- Evaluating vilm/vietcuna-3b-v2 (Few_Shot_VI_ViMedAQA) ---

--- Evaluating vilm/vietcuna-3b-v2 (Expert_Persona_VI) ---

--- Evaluating arcee-ai/Arcee-VyLinh (Direct_VI) ---

--- Evaluating arcee-ai/Arcee-VyLinh (RolePlay_VI) ---

--- Evaluating arcee-ai/Arcee-VyLinh (Extract_VI) ---

--- Evaluating a

,Model & Prompt Strategy,ROUGE-L,BLEU,METEOR,BERTScore-F1,Generation Time (s)
0,arcee-ai/Arcee-VyLinh (Extract_VI),0.6214,0.4536,0.5896,0.8551,754.84
1,arcee-ai/Arcee-VyLinh (Current_Best_VI),0.5832,0.3539,0.5556,0.8365,897.20
2,vilm/vietcuna-3b-v2 (Extract_VI),0.5403,0.2652,0.5315,0.8300,556.98
3,vilm/vietcuna-3b-v2 (Current_Best_VI),0.5333,0.2617,0.5413,0.8299,595.47
4,vilm/vietcuna-3b-v2 (Expert_Persona_VI),0.5284,0.2444,0.5415,0.8274,578.64
5,vilm/vietcuna-3b-v2 (RolePlay_VI),0.5195,0.2326,0.5312,0.8247,595.33
6,vilm/vietcuna-3b-v2 (Direct_VI),0.5167,0.2464,0.5123,0.8229,557.12
7,arcee-ai/Arcee-VyLinh (Few_Shot_VI_ViMedAQA),0.5262,0.2083,0.6351,0.8207,1777.16
8,vilm/vietcuna-3b-v2 (Chain_of_Thought_VI),0.5061,0.2063,0.5275,0.8175,674.53
9,alpha-ai/LLAMA3-3B-Medical-COT (Current_Best_VI),0.4925,0.2736,0.4527,0.8124,307.17
